In [17]:
pip install tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 4.9 MB/s eta 0:00:00


In [60]:
import tempfile
import os
from keras.models import load_model
import tensorflow as tf
import numpy as np

from tensorflow import keras

In [61]:
import random
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras import regularizers


In [63]:
train_dir = './train/'
test_dir = './test/'

train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   validation_split=0.2)

training_set = train_datagen.flow_from_directory(train_dir,
                                                batch_size=64,
                                                target_size=(48,48),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical',
                                                subset='training')
validation_set = train_datagen.flow_from_directory(train_dir,
                                                batch_size=64,
                                                target_size=(48,48),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical',
                                                subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True)

test_set = test_datagen.flow_from_directory(test_dir,
                                                batch_size=64,
                                                target_size=(48,48),
                                                shuffle=True,
                                                color_mode='grayscale',
                                                class_mode='categorical')

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [64]:
checkpoint_path = "fer_model.h5"
model = load_model(checkpoint_path)

weight_decay = 1e-4
num_classes = 7

In [66]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

initial_sparsity = 0.5
final_sparsity = 0.8

num_train_samples = training_set.samples

batch_size = 32
num_epochs = 10

steps_per_epoch = np.ceil(num_train_samples / batch_size).astype(np.int32)

begin_step = 0
end_step = steps_per_epoch * num_epochs


pruning_schedule = tfmot.sparsity.keras.PolynomialDecay(
    initial_sparsity=initial_sparsity,
    final_sparsity=final_sparsity,
    begin_step=begin_step,
    end_step=end_step
)

model_for_pruning = prune_low_magnitude(model, pruning_schedule=pruning_schedule)

callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]

model_for_pruning.compile(loss='categorical_crossentropy', optimizer=Adam(0.0003), metrics=['accuracy'])
model_for_pruning.fit(training_set, epochs=5, validation_data=validation_set, callbacks=callbacks)


Epoch 1/5
359/359 [==============================] - 32s 66ms/step - loss: 0.4718 - accuracy: 0.8673 - val_loss: 1.5378 - val_accuracy: 0.6187
Epoch 2/5
359/359 [==============================] - 24s 66ms/step - loss: 0.4831 - accuracy: 0.8627 - val_loss: 1.4390 - val_accuracy: 0.6152
Epoch 3/5
359/359 [==============================] - 23s 63ms/step - loss: 0.4976 - accuracy: 0.8595 - val_loss: 1.4868 - val_accuracy: 0.6184
Epoch 4/5
359/359 [==============================] - 23s 63ms/step - loss: 0.5053 - accuracy: 0.8542 - val_loss: 1.4589 - val_accuracy: 0.6234
Epoch 5/5
359/359 [==============================] - 23s 65ms/step - loss: 0.5101 - accuracy: 0.8501 - val_loss: 1.4264 - val_accuracy: 0.6208


In [67]:
print(f"Test accuracy = {model_for_pruning.evaluate(test_set ,batch_size=test_set.batch_size,steps=test_set.n // test_set.batch_size)[1]*100}%")

112/112 [==============================] - 3s 25ms/step - loss: 1.3898 - accuracy: 0.6313
Test accuracy = 63.12779188156128%


In [68]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.save('pruned_model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('pruned_fer_model.tflite', 'wb') as f:
    f.write(tflite_model)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [69]:
model = tf.keras.models.load_model('fer_model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('fer_model.tflite', 'wb') as f:
  f.write(tflite_model)

def get_tflite_model_size(file_path: str) -> int:
    size_bytes = os.path.getsize(file_path)
    return size_bytes

original_model_size = get_tflite_model_size('fer_model.tflite')
pruned_model_size = get_tflite_model_size('pruned_fer_model.tflite')
print(f"Original model size: {original_model_size} bytes")
print(f"Pruned model size: {pruned_model_size} bytes")

print(f"Pruned Ratio = {(1-pruned_model_size/original_model_size)*100}%")

Original model size: 5266152 bytes
Pruned model size: 1337264 bytes
Pruned Ratio = 74.6064298941618%
